## 👟👟 Passo a passo simples com unsloth: Fine-Tuning de LLMs

**Notebook simples**, passo a passo, com a escolha de um modelo, funções de Chat-Template para predição e treinamento, testes de predição antes e depois do treinamento.

📚 O objetivo desse notebook é aprender um pouco mais sobre o fine tuning de LLMs, não é ser um código final de treinamento.

Tudo começa com a escolha do modelo e a criação de um bom dataset de treinamento e, nesse exemplo, uma pergunta alvo que espera-se que o modelo aprenda: "O que é PUIL e qual o número da lei que criou o PUIL?"

### 🔗 Inspirado no exemplo de MariyaSha
  O que difere, ela fez uma versão bem compacta, não explorou chat template e não se preocupou com pads e tokens especiais de cada modelo, nem máscaras de atenção, facilitando o entendimento para quem está começando.
  Essa versão ainda difere por utilizar Unsloth para economizar memória e custo computacional.
  - Github: https://github.com/MariyaSha/fine_tuning
  - Vìdeo: https://youtu.be/uikZs6y0qgI?si=w6rOopXFyh7UxHbM

In [ ]:
#@title Instalando Unsloth
# https://docs.unsloth.ai/get-started/unsloth-notebooks
import os, re
from IPython.display import clear_output
try:
  import unsloth
  print("✅ Unsloth e vllm OK _o/")
  import transformers
  print("✅ Transformers OK _o/")
except ImportError as e:
    clear_output()
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
    !pip install transformers==4.55.4
    !pip install --no-deps trl==0.22.2
    clear_output()
    print('✅ Instalação do Unsloth e Transformers concluídas _o/')

✅ Instalação do Unsloth e Transformers concluídas _o/


# Escolhendo o modelo e preparando as classes

In [ ]:
#@title Escolhendo o modelo
# o comentário de acerto está relacionado à pergunta:
# - "O que é PUIL e qual o número da lei que criou o PUIL?"
model_name = 'meta-llama/Llama-3.2-1B-Instruct' # todo
model_name = 'meta-llama/Llama-3.2-3B-Instruct' # todo
model_name = 'google/gemma-3-27b-it' # 39Gb     # todo

model_name = "Jurema-br/Jurema-7B"       # 14Gb PUIL acertou em 50 épocas
model_name = "Qwen/Qwen2.5-3B-Instruct"  # PUIL 50 épocas - acertou a sigla e errou o resto
model_name = 'google/gemma-3-4b-it'      # 9Gb - PUIL errou tudo
model_name = "Qwen/Qwen2.5-7B-Instruct-1M" # 14Gb PUIL 50 épocas acertou a sigla e errou o resto
model_name = 'Qwen/Qwen3-8B' # 30Gb PUIL 50 épocas acertou sigla e lei
model_name ='deepseek-ai/DeepSeek-R1-Distill-Llama-8B' # 21Gb PUIL 50 épocas quase a sigla e quase a explicação
model_name = 'google/gemma-3-1b-it'   # 4.3Gb PUIL 50 épocas quase a sigla e quase a explicação/ 200 Sigla ok explicação quase, lei errada
model_name = 'google/gemma-3-12b-it'  # 25Gb PUIL 50 épocas errou tudo
model_name = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B' # 7.3Gb PUIL 50 épocas errou tudo / 300 continuou errando até o ptbr
model_name = 'google/gemma-3-4b-it'   # 9Gb - PUIL 50 épocas errou tudo


In [ ]:
#@title Prepara a classe de predição
''' Essa classe facilita a carga do modelo para predição antes e depois do treinamento
'''
print('Importando unsloth e transformers... ')
import os
import unsloth
from unsloth import FastModel
from unsloth.chat_templates import get_chat_template, CHAT_TEMPLATES
from transformers import GenerationConfig

class Modelo:
    ''' pode receber:
        - o nome do modelo para baixar
        - o nome da pasta com os adaptadores
        - uma tupla model, tokenizer
    '''
    def __init__(self, model_name:str, max_seq_length = 2048, n_bits = 8, cache_dir=None):
        self.model_name = model_name
        if isinstance(model_name, tuple):
            # é um modelo e um tokenizer em memória?
            self.model = model_name[0]
            self.tokenizer = model_name[1]
            self.model_name = 'Modelo em memória'
        else:
            # é um modelo em uma pasta ou o nome no huggingface
            bits4 = True if isinstance(n_bits, int) and n_bits == 4 else False
            bits8 = True if isinstance(n_bits, int) and n_bits == 8 else False
            arq_lora = 'adapter_config.json'
            arq_lora = os.path.join(model_name, arq_lora)
            self.lora_carregado = os.path.isfile(arq_lora)
            self.model, self.tokenizer = FastModel.from_pretrained(
                    model_name = model_name,
                    max_seq_length = max_seq_length, # Choose any for long context!
                    load_in_4bit = bits4,  # 4 bit quantization to reduce memory
                    load_in_8bit = bits8, # [NEW!] A bit more accurate, uses 2x memory
                    full_finetuning = False, # [NEW!] We have full finetuning now!
                    fast_inference = False,
                    device_map      = "auto",
                    cache_dir       = cache_dir,
            )
            self._ensure_chat_template()
        self.template_com_type(self.tokenizer)
        print('=' * 40)
        _com_lora = ' com LoRA' if self.lora_carregado else ''
        print(f'Modelo "{self.model_name}"{_com_lora} carregado!')

    def _ensure_chat_template(self):
        if getattr(self.tokenizer, "chat_template", None):
            return  # já tem template definido (modelos "instruct" costumam trazer)
        _nm_teste = self.model_name.replace('-','')
        self.message_use_type = False
        if 'gemma' in _nm_teste:
            key = 'gemma'
            self.message_use_type = True
        elif 'qwen2' in _nm_teste: key = '"qwen-2.5"'
        elif 'qwen3' in _nm_teste: key = 'chatml'
        elif 'llama33' in _nm_teste:  key = 'llama-3.3'
        elif 'llama32' in _nm_teste:  key = 'llama-3.2'
        elif 'llama31' in _nm_teste:  key = 'llama-3.1'
        elif 'llama3' in _nm_teste:  key = 'llama-3'
        elif 'llama' in _nm_teste:  key = 'llama'
        else: key = 'chatml'
        if key not in CHAT_TEMPLATES:
            key = "chatml"  # último fallback
        self.tokenizer = get_chat_template(self.tokenizer, chat_template=key)

    def _place_inputs(self, inputs):
        #if self._is_sharded():
        #    return inputs#.to("cpu") # deixa o acelerate decidir o device correto
        try:
            target = self.model.device
        except AttributeError:
            target = next(self.model.parameters()).device
        return inputs.to(target)

    @classmethod
    def template_com_type(self, tokenizer) -> bool:
        msgs_list = [{"role": "user", "content": [{"type":"text","text":"ping"}]}]
        msgs_str  = [{"role": "user", "content": "ping"}]
        try:
            tokenizer.apply_chat_template(msgs_list, tokenize=False)
            self._msg_type = True
            return  True # lista de partes funcionou
        except Exception:
            pass
        # se lista falhou, tente string
        tokenizer.apply_chat_template(msgs_str, tokenize=False)  # lança se não suportar
        self._msg_type = False
        return False # string funcionou

    def resposta(self, prompt):
        content = [{"type": "text", "text": prompt}] if self._msg_type else prompt
        messages = [{"role": "user", "content": content}]
        # Formata a conversa conforme o template e prepara tensores
        inputs = self.tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,  # requerido para inferência/chat
            return_tensors="pt",
        )
        inputs = self._place_inputs(inputs)
        #inputs = {k: v.to(device) for k, v in inputs.items()}

        # Config de geração (ajuste conforme necessidade)
        base_cfg = GenerationConfig(
            max_new_tokens=256,
            do_sample=False,           # determinístico por padrão
            temperature=0.0,
            top_p=1.0,
            pad_token_id=self.tokenizer.eos_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
        )
        #if generation_kwargs:
        #    base_cfg = GenerationConfig(**{**base_cfg.to_dict(), **generation_kwargs})

        # Geração
        outputs = self.model.generate(inputs, generation_config=base_cfg)

        # Decodifica apenas o trecho gerado (exclui o prompt)
        #prompt_len = inputs["input_ids"].shape[-1]
        prompt_len = inputs.shape[-1]
        generated = outputs[0][prompt_len:]
        return self.tokenizer.decode(generated, skip_special_tokens=True).strip()

    def print_pergunta_resposta(self, prompt):
        print('=' * 50)
        print('Pergunta:', prompt)
        print('Resposta do modelo:', self.resposta(prompt))
        print('-' * 50)

print('✅ Tudo ok!')

Importando unsloth e transformers... 
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
✅ Tudo ok!


In [ ]:
#@title Carga e predição do modelo base

# ATENÇÃO: se tiver pouca memória de GPU, reinicie a sessão e pule essa célula para
#          realizar o treinamento com a GPU livre

md = Modelo(model_name)
md.print_pergunta_resposta("O que é a classe processual PUIL?")


# Dataset

Precisamos ensinar o modelo com prompts e suas respectivas respostas. Podemos usar um dataset padrão para a entrada, mas cada modelo tem um formato próprio que pode ser formadado com o chat-template do modelo:

### Formato padrão de entrada
- JsonL Um json por linha
```
{"prompt": "O que é PUIL?", "completion": "Uma classe processual do STJ"}
{"prompt": "O que significa PUIL", "completion": "Pedido de Uniformização de Interpretação de Lei"}
```


In [ ]:
#@title Passo 1: carregando dados para o treino
from datasets import load_dataset
arq = "puil_treinamento.txt"
#carrega do git
if not os.path.isfile(arq):
   ! curl -o {arq} https://raw.githubusercontent.com/luizanisio/llms/refs/heads/main/ntb_treinamento/puil_treinamento.txt
raw_data = load_dataset("json", data_files=arq)
print('Dataset carregado:', raw_data)

print('Exemplo:', raw_data["train"][0])

In [ ]:
#@title Passo 2 processando dados com chat template
from transformers import AutoTokenizer
from datasets import load_dataset # Exemplo, caso precise carregar um dataset

''' Alguns modelos de instrução não têm um pad_token, se for o caso o pad_token
    será configurado para usar o eos_token.
    Para otimizar o SFT e apontar os mecanismos de atenção apenas para a predição,
    os labels e máscaras de atenção vão ser configurados para que o treinamento
    não perca tempo com os tokens do prompt, epenas aprendam com a resposta.
    Aqui faz-se um workaround para identificar exatamente os tokens que serão aprendidos
    ao aplicar o chat template sem a resposta e com a resposta.
    Poderiam ser identificados os tokens especiais de cada modelo, mas perderíamos a
    chance de utilizar o chat_template próprio que já absorve essa inteligência para cada
    modelo.
'''
print('Preparando dataset com o modelo:', model_name)

train_tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

if train_tokenizer.pad_token is None:
    print('ATENÇÃO: O modelo não tinha pad_token e foi utilizado o eos_token')
    train_tokenizer.pad_token = train_tokenizer.eos_token
usar_type = Modelo.template_com_type(train_tokenizer)

def preprocess_with_chat_template(sample, max_length=1024):
    """
    Mascara tudo que vem antes do início da resposta do assistant e todo padding.
    Compatível com chat template do Gemma (sem procurar tags em string).
    """
    # 1) IDs até o ponto em que o modelo começaria a responder (sem completion):
    content_usr = [{"type": "text", "text": sample["prompt"]}] if usar_type else sample["prompt"]
    content_ast = [{"type": "text", "text": sample["completion"]}] if usar_type else sample["completion"]
    messages_prompt = [{"role": "user", "content": content_usr},]
    prompt_ids = train_tokenizer.apply_chat_template(
        messages_prompt,
        tokenize=True,
        add_generation_prompt=True,   # adiciona o cabeçalho do assistant/model
        return_tensors=None,
    )

    # 2) IDs da conversa completa (com a resposta do assistant):
    messages_full = [
        {"role": "user", "content": content_usr},
        {"role": "assistant", "content": content_ast}, ]

    full_ids = train_tokenizer.apply_chat_template(
        messages_full,
        tokenize=True,
        add_generation_prompt=False,
        return_tensors=None,
    )

    # 3) Truncar/padding de forma consistente
    # Observação: truncando manualmente para preservar o cutoff corretamente.
    input_ids = full_ids[:max_length]
    attention_mask = [1] * len(input_ids)
    if len(input_ids) < max_length:
        pad_len = max_length - len(input_ids)
        input_ids = input_ids + [train_tokenizer.pad_token_id] * pad_len
        attention_mask = attention_mask + [0] * pad_len

    # 4) Cutoff = início da resposta do assistant (limitado pelo max_length)
    cutoff = min(len(prompt_ids), max_length)

    # 5) Labels = cópia de input_ids; máscara em [0:cutoff) e em padding
    labels = input_ids.copy()
    for j in range(max_length):
        if j < cutoff or attention_mask[j] == 0:
            labels[j] = -100 # labels para serem ignorados no treinamento

    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
    }

def calcular_qtd_tokens(sample):
    messages_full = [
        {"role": "user", "content": sample["prompt"]},
        {"role": "assistant", "content": sample["completion"]},]
    full_ids = train_tokenizer.apply_chat_template(
        messages_full,
        tokenize=True,
        add_generation_prompt=False,
        return_tensors=None,
    )
    return {'qtd_tokens':len(full_ids)}

print('Colunas originais:',raw_data.column_names)
print(raw_data.column_names['train'][0])
# --- Processando o Dataset ---
# Inicialmente vamos calcular o número máximo de de tokens que o dataset exige
qtd_tokens_dataset = raw_data.map(
    calcular_qtd_tokens,
    remove_columns=raw_data.column_names['train']
)
min_tokens = min(qtd_tokens_dataset['train']['qtd_tokens'])
max_tokens = max(qtd_tokens_dataset['train']['qtd_tokens'])
print('Mínimo de tokens identificados no dataset: ', min_tokens)
print('Máximo de tokens identificados no dataset: ', max_tokens)
# Aplicamos a nova função ao seu conjunto de dados.
# `remove_columns` é útil para limpar as colunas de texto originais que não são mais necessárias.
data = raw_data.map(
    lambda x: preprocess_with_chat_template(x, max_length=max_tokens),
    remove_columns=raw_data.column_names['train']
)
print('Colunas treino:', data.column_names)

i_teste = 0
# Vamos decodificar uma amostra para ver como ficou
print("Um exemplo processado:")
print(data['train'][i_teste])
print("\nAmostra decodificada:")
print(train_tokenizer.decode(data['train'][i_teste]['input_ids']))





In [ ]:
#@title Quais módulos do modelo podem ser treinados
''' O treinamento pode impactar vários módulos do modelo,
    para os exemplos mais comuns, usa-se os módulos ['k_proj','q_proj', 'v_proj']
    mas pode-se usar todos os módulos lineares.
'''

def modulos_treino(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        # A camada que queremos treinar pode ser uma camada linear padrão (torch.nn.Linear)
        # ou uma camada linear quantizada (como a do bitsandbytes).
        # O ideal é verificar por um nome de classe que abranja ambos, como 'Linear'.
        if 'Linear' in str(type(module)):
            # Os nomes dos módulos LoRA são geralmente os últimos componentes do nome completo.
            # Ex: "model.layers.0.self_attn.q_proj" -> queremos "q_proj"
            module_name = name.split('.')[-1]
            lora_module_names.add(module_name)

    print("\n" + "="*50)
    print("Módulos lineares encontrados que podem ser usados como `target_modules` para LoRA:")
    print(lora_module_names)
    print("="*50)
    return lora_module_names


In [ ]:
#@title Passo 3 preparando LoRa
from unsloth import FastLanguageModel
from peft import PeftModel

print('MODEL NAME:', md.model_name)
print('Tipo do modelo antes de aplicar o Peft', type(md.model))

target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",]

train_model = FastLanguageModel.get_peft_model(
    md.model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = target_modules,
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


print('Tipo do modelo depois de aplicar o Peft', type(train_model))
print('É um modelo Peft?', isinstance(train_model, PeftModel))
print('LoRA preparado para treinar os módulos:', target_modules)

MODEL NAME: google/gemma-3-4b-it
Tipo do modelo antes de aplicar o Peft <class 'transformers.models.gemma3.modeling_gemma3.Gemma3ForConditionalGeneration'>
Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients
Tipo do modelo depois de aplicar o Peft <class 'peft.peft_model.PeftModelForCausalLM'>
É um modelo Peft? True
LoRA preparado para treinar os módulos: ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']


## 4 e 5 - Treinamento propriamente dito

- defina o número de épocas, decaimento do learning rate, aquecimento do learning rate, localização dos logs de treinamento, etc

In [ ]:
#@title Passo 4 preparando um treino simples
# https://huggingface.co/docs/trl/en/sft_trainer

from trl import SFTTrainer, SFTConfig
EPOCAS = 50

trainer = SFTTrainer(
    model = train_model,              # modelo já carregado (HF/Unsloth/PEFT)
#   tokenizer = tokenizer,            # desnecessário pois o dataset jé retorna os números dos tokens
    train_dataset = data["train"],    # dataset de treino com o campo de texto abaixo
    eval_dataset  = None,             # defina um dataset de validação p/ métricas periódicas
    args = SFTConfig(
        num_train_epochs = EPOCAS,    # epochs de treino
        dataset_text_field = "text",  # coluna do dataset que contém o texto de entrada
        per_device_train_batch_size = 2,   # batch por GPU/TPU (antes de acumulação)
        gradient_accumulation_steps = 4,   # nº de passos acumulados antes do otimizador dar step
                                           # ⇒ batch_efetivo ≈ batch_size * acum_steps * nº_dispositivos
        warmup_steps = 5,             # passos iniciais de aquecimento do LR (cresce do 0 ao LR alvo)
        learning_rate = 2e-4,         # taxa inicial; use menor (ex.: 2e-5) p/ treinos longos/estáveis
        optim = "adamw_8bit",         # AdamW com estados em 8-bit (economia de VRAM, finetuning de LLMs)
        weight_decay = 0.01,          # regularização L2 nos pesos (evita overfitting em cabeças densas)
        lr_scheduler_type = "linear", # LR decai linearmente após warmup (padrão no HF Trainer)
        seed = 3407,                  # reprodutibilidade (shuffles, inicialização, etc.)
        report_to = "none",           # "wandb", "tensorboard", etc. p/ enviar métricas
        logging_steps=25,
        logging_dir='./log',
        output_dir='./results' # Output directory for saving checkpoints and logs
    ),
)

In [ ]:
#@title Passo 4.1 treinando
trainer_stats = trainer.train()

In [ ]:
#@title Passo 5 testando o modelo treinado em memória

#md = Modelo((train_model, train_tokenizer))
md.print_pergunta_resposta("O que é PUIL e qual o número da lei que criou o PUIL?")

Pergunta: O que é PUIL e qual o número da lei que criou o PUIL?
Resposta do modelo: É o Pedido de Uniformização de Interpretação de Lei; a lei é a TNU (art. 14).
--------------------------------------------------


# Gravando o modelo no disco
- você precisa salvar o modelo e o tokenizer juntos para serem carregados no futuro

In [ ]:
# só os pesos do LoRA
print('Gravando pesos do LoRA')
train_model.save_pretrained("model_lora")  # Local saving
train_tokenizer.save_pretrained("model_lora")

print('Gravando merge 16bit')
train_model.save_pretrained_merged("model_merged", train_tokenizer, save_method = "merged_16bit",)

print('Gravando base e LoRA')
# modelo e LoRA
trainer.save_model("./model")
train_tokenizer.save_pretrained("./model")


## Carregando o modelo do disco para teste
- Nesse ponto pode ser interessante reiniciar a sessão do notebook para liberar memória e carregar apenas o modelo treinado


In [ ]:
md = Modelo('./model_merged')
md.print_pergunta_resposta("O que é PUIL e qual o número da lei que criou o PUIL?")

In [ ]:
#@title Carregando do disco e testando o modelo base
md = Modelo(model_name)
md.print_pergunta_resposta("O que é PUIL e qual o número da lei que criou o PUIL?")